In [ ]:
import jax
import jax.numpy as jnp
import jax_ppo
import matplotlib.pyplot as plt
import optax

In [ ]:
import flock_env

In [ ]:
k = jax.random.PRNGKey(101)

## Hyper-Parameters

In [ ]:
n_agents = 20
n_train = 1_500
n_train_env = 32
n_test_env = 5
n_env_steps = 200
n_train_epochs = 2
mini_batch_size = 256

## Training Environment

In [ ]:
env = flock_env.SimpleFlockEnv(
    reward_func=flock_env.rewards.exponential_rewards, n_agents=n_agents
)
env_params = env.default_params

In [ ]:
n_steps = n_train * n_train_env * n_env_steps * n_train_epochs // mini_batch_size

## PPO Agent

In [ ]:
params = jax_ppo.default_params._replace(
    gamma=0.99, 
    gae_lambda=0.99, 
    entropy_coeff=0.002, 
    adam_eps=1e-8, 
    clip_coeff=0.2
)

In [ ]:
train_schedule = optax.linear_schedule(2e-3, 2e-5, n_steps)

In [ ]:
k, agent = jax_ppo.init_agent(
    k,
    params,
    env.action_space(env_params).shape,
    env.observation_space(env_params).shape,
    train_schedule,
    layer_width=16,
)

## Train Agent

In [ ]:
k, trained_agent, losses, state_ts, rewards, _ = jax_ppo.train(
    k,
    env,
    env_params,
    agent,
    n_train,
    n_train_env,
    n_train_epochs,
    mini_batch_size,
    n_test_env,
    params,
    greedy_test_policy=True,
    n_env_steps=n_env_steps,
    n_agents=n_agents,
)

In [ ]:
plt.plot(jnp.mean(jnp.sum(rewards, axis=(2, 3)), axis=1));
plt.xlabel("Train Step")
plt.ylabel("Avg Total Rewards");

In [ ]:
import matplotlib
matplotlib.use('Agg')
matplotlib.rc('animation', html='html5')

In [ ]:
anim = flock_env.visualisation.animate_agents(
    state_ts.agent_positions[-1, 0],
    state_ts.agent_headings[-1, 0],
    rewards[-1, 0]
)

In [ ]:
anim